# Single Video

In [5]:
import os
import yt_dlp

def download_video_with_audio():
    """
    Downloads the video based on the following conditions:
    - If 720p video with audio is available, download it.
    - If 720p is not available, download 1080p with audio.
    - If neither 720p nor 1080p are available, download the highest available resolution with audio.
    - Create a folder inside 'output_directory' using the video title and save the downloaded video inside that folder.
    """
    # Get YouTube URL from the user
    video_url = input("Enter the YouTube video URL: ").strip()
    
    # Directory to save downloaded files
    output_directory = "downloads"
    
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    try:
        with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
            # Extract video info without downloading
            info_dict = ydl.extract_info(video_url, download=False)
            video_title = info_dict.get('title', 'Unnamed Video')
            formats = info_dict.get('formats', [])

            # Create a folder using the video title
            video_folder = os.path.join(output_directory, video_title)
            os.makedirs(video_folder, exist_ok=True)
            
            # Video format preferences (720p and 1080p)
            preferred_formats = {
                '720': '(bv*[height=720]+ba)',  # 720p with audio
                '1080': '(bv*[height=1080]+ba)',  # 1080p with audio
            }

            # Check for 720p or 1080p video availability
            is_720p_available = any(f.get('height') == 720 for f in formats)
            is_1080p_available = any(f.get('height') == 1080 for f in formats)

            if is_720p_available:
                print("720p with audio available. Downloading...")
                ydl_opts = {
                    'format': preferred_formats['720'],
                    'merge_output_format': 'mp4',
                    'outtmpl': os.path.join(video_folder, '%(title)s.%(ext)s'),
                }
            elif is_1080p_available:
                print("720p not available. Downloading 1080p with audio...")
                ydl_opts = {
                    'format': preferred_formats['1080'],
                    'merge_output_format': 'mp4',
                    'outtmpl': os.path.join(video_folder, '%(title)s.%(ext)s'),
                }
            else:
                # If neither 720p nor 1080p are available, download the highest available resolution with audio
                print("720p and 1080p are not available. Downloading the highest available resolution with audio...")
                highest_res_format = max(formats, key=lambda f: f.get('height', 0))  # Get highest resolution
                ydl_opts = {
                    'format': f"b{highest_res_format['format_id']}+ba",  # Highest resolution with audio
                    'merge_output_format': 'mp4',
                    'outtmpl': os.path.join(video_folder, '%(title)s.%(ext)s'),
                }

            # Download the selected video format
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([video_url])

            print(f"Download complete! The video has been saved in: {video_folder}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Run the function
download_video_with_audio()


Enter the YouTube video URL:  https://www.youtube.com/watch?v=dwA24nEGohM


720p with audio available. Downloading...
[youtube] Extracting URL: https://www.youtube.com/watch?v=dwA24nEGohM
[youtube] dwA24nEGohM: Downloading webpage
[youtube] dwA24nEGohM: Downloading ios player API JSON
[youtube] dwA24nEGohM: Downloading mweb player API JSON
[youtube] dwA24nEGohM: Downloading m3u8 information
[info] dwA24nEGohM: Downloading 1 format(s): 302+251
[download] Destination: downloads/Anil Ki Paneer Chaat लाजवाब 😋 । Achanak Se Kaha Jaana Pada 😮 | Rakesh's Life |/Anil Ki Paneer Chaat लाजवाब 😋 । Achanak Se Kaha Jaana Pada 😮 ｜ Rakesh's Life ｜.f302.webm
[download] 100% of  253.51MiB in 00:01:21 at 3.11MiB/s      
[download] Destination: downloads/Anil Ki Paneer Chaat लाजवाब 😋 । Achanak Se Kaha Jaana Pada 😮 | Rakesh's Life |/Anil Ki Paneer Chaat लाजवाब 😋 । Achanak Se Kaha Jaana Pada 😮 ｜ Rakesh's Life ｜.f251.webm
[download] 100% of   12.88MiB in 00:00:04 at 2.88MiB/s   
[Merger] Merging formats into "downloads/Anil Ki Paneer Chaat लाजवाब 😋 । Achanak Se Kaha Jaana Pada 😮 | Ra

# Playlist Video Download

In [ ]:
import os
import yt_dlp

def download_playlist_with_audio(playlist_url, output_directory="downloads"):
    """
    Downloads a YouTube playlist with the specified conditions:
    - Downloads 720p video with audio if available.
    - If 720p is not available, downloads 1080p with audio.
    - If neither is available, downloads the highest resolution with audio.
    - Saves all videos in a folder named after the playlist title.

    Args:
        playlist_url (str): The YouTube playlist URL.
        output_directory (str): The base directory to save the playlist.
    """
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    def progress_hook(d):
        """Display download progress."""
        if d['status'] == 'finished':
            print(f"Downloaded: {d['filename']}")

    ydl_opts = {
        'quiet': False,
        'format': '(bv*[height=720]+ba/bv*[height=1080]+ba/bv+ba/b)',
        'merge_output_format': 'mp4',
        'progress_hooks': [progress_hook],
        'outtmpl': f'{output_directory}/%(playlist_title)s/%(playlist_index)s - %(title)s.%(ext)s',  # Folder for playlist
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',
        }],
        'skip_download': False,  # Ensure videos are downloaded
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(playlist_url, download=False)
            if "entries" not in info_dict:
                print("No videos found in the playlist!")
                return

            playlist_title = info_dict.get('title', 'Unnamed Playlist')
            print(f"Downloading playlist: {playlist_title}")

            # Download videos in the playlist
            ydl.download([playlist_url])

        print(f"All videos from '{playlist_title}' have been downloaded!")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
playlist_url = input("Enter the YouTube playlist URL: ").strip()
download_playlist_with_audio(playlist_url)


Enter the YouTube playlist URL:  https://www.youtube.com/watch?v=BP-w99ZINTc&list=PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6&index=1&t=5s


[youtube:tab] Extracting URL: https://www.youtube.com/watch?v=BP-w99ZINTc&list=PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6&index=1&t=5s
[youtube:tab] Downloading playlist PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 - add --no-playlist to download just the video BP-w99ZINTc
[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6: Downloading webpage
[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6
[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6: Downloading webpage
[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Generative AI Series - Showcasing Generative AI By Building Projects
[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 page 1: Downloading API JSON


[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 page 1: Downloading API JSON


[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 page 1: Downloading API JSON


[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 page 1: Downloading API JSON


[youtube:tab] Playlist Generative AI Series - Showcasing Generative AI By Building Projects: Downloading 6 items of 6
[download] Downloading item 1 of 6
[youtube] Extracting URL: https://www.youtube.com/watch?v=BP-w99ZINTc
[youtube] BP-w99ZINTc: Downloading webpage
[youtube] BP-w99ZINTc: Downloading ios player API JSON
[youtube] BP-w99ZINTc: Downloading mweb player API JSON
[youtube] BP-w99ZINTc: Downloading m3u8 information
[download] Downloading item 2 of 6
[youtube] Extracting URL: https://www.youtube.com/watch?v=dGdA9z6gBkM
[youtube] dGdA9z6gBkM: Downloading webpage
[youtube] dGdA9z6gBkM: Downloading ios player API JSON
[youtube] dGdA9z6gBkM: Downloading mweb player API JSON
[youtube] dGdA9z6gBkM: Downloading m3u8 information
[download] Downloading item 3 of 6
[youtube] Extracting URL: https://www.youtube.com/watch?v=cQUUkZnyoD0
[youtube] cQUUkZnyoD0: Downloading webpage
[youtube] cQUUkZnyoD0: Downloading ios player API JSON
[youtube] cQUUkZnyoD0: Downloading mweb player API JSON


# Combined

In [1]:
import os
import yt_dlp

def detect_url_type(url):
    """
    Detects if a URL is for a single video or a playlist.
    
    Args:
        url (str): The YouTube URL to check.
    
    Returns:
        str: "single video" if it's a video URL, "playlist" if it's a playlist URL.
    """
    try:
        # Initialize yt-dlp with options
        ydl_opts = {'quiet': True}
        
        # Extract video information without downloading
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=False)
            
            # Check if it's a playlist or a single video
            if 'entries' in info_dict:
                if len(info_dict['entries']) > 1:
                    return "playlist"
                else:
                    return "single video"
            else:
                return "single video"
    except Exception as e:
        print(f"Error: {e}")
        return "Invalid URL"


def download_video_with_audio(video_url, output_directory="downloads"):
    """
    Downloads a single YouTube video with audio based on the resolution available.
    
    Args:
        video_url (str): The YouTube video URL.
        output_directory (str): Directory to save the downloaded video.
    """
    # Directory to save downloaded files
    os.makedirs(output_directory, exist_ok=True)

    try:
        with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
            # Extract video info without downloading
            info_dict = ydl.extract_info(video_url, download=False)
            video_title = info_dict.get('title', 'Unnamed Video')
            formats = info_dict.get('formats', [])

            # Create a folder using the video title
            video_folder = os.path.join(output_directory, video_title)
            os.makedirs(video_folder, exist_ok=True)
            
            # Video format preferences (720p and 1080p)
            preferred_formats = {
                '720': '(bv*[height=720]+ba)',  # 720p with audio
                '1080': '(bv*[height=1080]+ba)',  # 1080p with audio
            }

            # Check for 720p or 1080p video availability
            is_720p_available = any(f.get('height') == 720 for f in formats)
            is_1080p_available = any(f.get('height') == 1080 for f in formats)

            if is_720p_available:
                print("720p with audio available. Downloading...")
                ydl_opts = {
                    'format': preferred_formats['720'],
                    'merge_output_format': 'mp4',
                    'outtmpl': os.path.join(video_folder, '%(title)s.%(ext)s'),
                }
            elif is_1080p_available:
                print("720p not available. Downloading 1080p with audio...")
                ydl_opts = {
                    'format': preferred_formats['1080'],
                    'merge_output_format': 'mp4',
                    'outtmpl': os.path.join(video_folder, '%(title)s.%(ext)s'),
                }
            else:
                # If neither 720p nor 1080p are available, download the highest available resolution with audio
                print("720p and 1080p are not available. Downloading the highest available resolution with audio...")
                highest_res_format = max(formats, key=lambda f: f.get('height', 0))  # Get highest resolution
                ydl_opts = {
                    'format': f"b{highest_res_format['format_id']}+ba",  # Highest resolution with audio
                    'merge_output_format': 'mp4',
                    'outtmpl': os.path.join(video_folder, '%(title)s.%(ext)s'),
                }

            # Download the selected video format
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([video_url])

            print(f"Download complete! The video has been saved in: {video_folder}")

    except Exception as e:
        print(f"An error occurred: {e}")


def download_playlist_with_audio(playlist_url, output_directory="downloads"):
    """
    Downloads a YouTube playlist with the specified conditions.
    
    Args:
        playlist_url (str): The YouTube playlist URL.
        output_directory (str): Directory to save the downloaded playlist.
    """
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    def progress_hook(d):
        """Display download progress."""
        if d['status'] == 'finished':
            print(f"Downloaded: {d['filename']}")

    ydl_opts = {
        'quiet': False,
        'format': '(bv*[height=720]+ba/bv*[height=1080]+ba/bv+ba/b)',  # Prefer 720p, then 1080p, then highest
        'merge_output_format': 'mp4',
        'progress_hooks': [progress_hook],
        'outtmpl': f'{output_directory}/%(playlist_title)s/%(playlist_index)s - %(title)s.%(ext)s',  # Folder for playlist
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',
        }],
        'skip_download': False,  # Ensure videos are downloaded
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(playlist_url, download=False)
            if "entries" not in info_dict:
                print("No videos found in the playlist!")
                return

            playlist_title = info_dict.get('title', 'Unnamed Playlist')
            print(f"Downloading playlist: {playlist_title}")

            # Download videos in the playlist
            ydl.download([playlist_url])

        print(f"All videos from '{playlist_title}' have been downloaded!")

    except Exception as e:
        print(f"An error occurred: {e}")


def main():
    # Get the URL from the user
    url = input("Enter the YouTube URL (single video or playlist): ").strip()

    # Detect whether the URL is for a single video or a playlist
    url_type = detect_url_type(url)
    
    if url_type == "single video":
        print("Detected single video URL.")
        download_video_with_audio(url)
    elif url_type == "playlist":
        print("Detected playlist URL.")
        download_playlist_with_audio(url)
    else:
        print("Invalid URL. Please enter a valid YouTube video or playlist URL.")


# Run the main function
if __name__ == "__main__":
    main()


Enter the YouTube URL (single video or playlist):  https://www.youtube.com/watch?v=BP-w99ZINTc&list=PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6&index=1&t=5s


Detected playlist URL.
[youtube:tab] Extracting URL: https://www.youtube.com/watch?v=BP-w99ZINTc&list=PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6&index=1&t=5s
[youtube:tab] Downloading playlist PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 - add --no-playlist to download just the video BP-w99ZINTc
[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6: Downloading webpage
[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6
[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6: Downloading webpage
[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Generative AI Series - Showcasing Generative AI By Building Projects
[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 page 1: Downloading API JSON


[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 page 1: Downloading API JSON


[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 page 1: Downloading API JSON


[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 page 1: Downloading API JSON


...

[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 page 1: Downloading API JSON


[youtube:tab] PLu0W_9lII9aiS4rUVp2jXwIvCruo27sG6 page 1: Downloading API JSON


[youtube:tab] Playlist Generative AI Series - Showcasing Generative AI By Building Projects: Downloading 6 items of 6
[download] Downloading item 1 of 6
[youtube] Extracting URL: https://www.youtube.com/watch?v=BP-w99ZINTc
[youtube] BP-w99ZINTc: Downloading webpage
[youtube] BP-w99ZINTc: Downloading ios player API JSON
[youtube] BP-w99ZINTc: Downloading mweb player API JSON
[youtube] BP-w99ZINTc: Downloading m3u8 information
[info] BP-w99ZINTc: Downloading 1 format(s): 302+251
[download] Destination: downloads/Generative AI Series - Showcasing Generative AI By Building Projects/1 - HarryGPT - An AI Based Discord Bot 🔥 ｜ Generative AI Series.f302.webm
[download] 100% of  136.86MiB in 00:01:42 at 1.34MiB/s     Downloaded: downloads/Generative AI Series - Showcasing Generative AI By Building Projects/1 - HarryGPT - An AI Based Discord Bot 🔥 ｜ Generative AI Series.f302.webm

[download] Destination: downloads/Generative AI Series - Showcasing Generative AI By Building Projects/1 - HarryGPT 

[download] Got error: HTTPSConnectionPool(host='rr3---sn-5jucgv5qc5oq-cagr.googlevideo.com', port=443): Read timed out.


[download] Got error: HTTPSConnectionPool(host='rr3---sn-5jucgv5qc5oq-cagr.googlevideo.com', port=443): Read timed out.
